In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from openai import OpenAI
import os
import re
import time
import csv
from collections import Counter

In [3]:
df = pd.read_csv('CNC_synth_copy2.csv')

In [4]:
df.head()

,Text
0,The company accused <ARG1>the employee of stea...
1,"""<ARG1>The students organized a protest</ARG1>..."
2,'<ARG1>The students are excited</ARG1> <SIG0>b...
3,'<ARG0>Poor air quality</ARG0> <SIG0>can <ARG1...
4,Since the news about the company's financial d...


In [7]:
def filter_dataframe(df, word_one, word_two):
    filtered_df = df[df['Text'].str.contains(word_one, case=False) & df['Text'].str.contains(word_two, case=False)]

    return filtered_df

In [8]:
#this deletes any entries that lack an ARG0 or an ARG1
df_f = filter_dataframe(df,'<ARG0>', '<ARG1>')

In [9]:
len(df_f)

966

In [10]:
def find_common_words(df):
    word_frequency = Counter()

    # Iterate through each entry in the DataFrame
    for index, row in df.iterrows():
        # Combine the entries into a single string
        combined_text = ' '.join(row)

        # Tokenize the text into words (you might need a more sophisticated tokenizer based on your data)
        words = re.findall(r'\b\w+\b', combined_text.lower())

        # Update word frequency based on the set of unique words in the entry
        word_frequency.update(set(words))

    # Get the most common words across all entries
    most_common_words = word_frequency.most_common()

    return most_common_words

In [13]:
#This is to get a statistic of the most common words used in each of the sentences. This will inform which entries will be cut from the final dataset.

common_words = find_common_words(df_f)
print(common_words)

[('arg1', 966), ('arg0', 966), ('sig0', 852), ('the', 852), ('to', 659), ('in', 537), ('and', 368), ('a', 362), ('of', 316), ('heavy', 282), ('due', 210), ('city', 169), ('caused', 158), ('flooding', 157), ('rain', 149), ('their', 113), ('for', 113), ('resulting', 112), ('widespread', 112), ('rainfall', 109), ('company', 101), ('new', 96), ('after', 95), ('because', 93), ('on', 91), ('as', 85), ('led', 85), ('increase', 84), ('s', 82), ('was', 77), ('were', 76), ('government', 74), ('students', 73), ('damage', 71), ('streets', 71), ('that', 68), ('causing', 67), ('has', 67), ('traffic', 65), ('flooded', 65), ('by', 63), ('lack', 58), ('severe', 58), ('leading', 56), ('residents', 53), ('several', 51), ('order', 51), ('levels', 51), ('increased', 50), ('pollution', 47), ('homes', 46), ('implemented', 46), ('with', 45), ('many', 45), ('she', 45), ('car', 45), ('infrastructure', 45), ('resulted', 44), ('have', 42), ('decided', 41), ('rise', 40), ('significant', 40), ('strong', 40), ('they

In [15]:
csv_file_path = 'CNC_train_raw.csv'
df_f.to_csv(csv_file_path, index=False)

In [18]:
#code block by shubhika


# Adding a new column 'sent_id' starting from 10000 and incrementing by 1
df_f['sent_id'] = range(10000, 10000 + len(df_f))

# Adding a new column 'corpus' with constant value 'cnc'
df_f['corpus'] = 'cnc'

# Adding a new column 'eg_id' with constant value 0
df_f['eg_id'] = 0

# Adding a new column 'doc_id' with values 'train_1000_10000' where 'train_' is constant, and the other two values are incremented by 1
df_f['doc_id'] = [f'train_{i}_{j}' for i, j in zip(range(1000, 1000 + len(df_f)), range(10000, 10000 + len(df_f)))]

# Adding a new column 'index' with values based on variables i, j, k
df_f['index'] = [f'cnc_train_{i}_{j}_{k}_10' for i, j, k in zip(range(100, 100 + len(df_f)), range(1000, 1000 + len(df_f)), range(10000, 10000 + len(df_f)))]


/var/folders/0z/7ky_dy791xqbq2pvq3wkjlmr0000gn/T/ipykernel_42126/223307239.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_f['sent_id'] = range(10000, 10000 + len(df_f))
/var/folders/0z/7ky_dy791xqbq2pvq3wkjlmr0000gn/T/ipykernel_42126/223307239.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_f['corpus'] = 'cnc'
/var/folders/0z/7ky_dy791xqbq2pvq3wkjlmr0000gn/T/ipykernel_42126/223307239.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .l

In [24]:
df_f.rename(columns={'Text': 'causal_text_w_pairs'}, inplace=True)

/var/folders/0z/7ky_dy791xqbq2pvq3wkjlmr0000gn/T/ipykernel_42126/3188732107.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_f.rename(columns={'Text': 'causal_text_w_pairs'}, inplace=True)


In [25]:
df_f.head()

,causal_text_w_pairs,sent_id,corpus,eg_id,doc_id,index
0,The company accused <ARG1>the employee of stea...,10000,cnc,0,train_1000_10000,cnc_train_100_1000_10000_10
1,"""<ARG1>The students organized a protest</ARG1>...",10001,cnc,0,train_1001_10001,cnc_train_101_1001_10001_10
2,'<ARG1>The students are excited</ARG1> <SIG0>b...,10002,cnc,0,train_1002_10002,cnc_train_102_1002_10002_10
3,'<ARG0>Poor air quality</ARG0> <SIG0>can <ARG1...,10003,cnc,0,train_1003_10003,cnc_train_103_1003_10003_10
4,Since the news about the company's financial d...,10004,cnc,0,train_1004_10004,cnc_train_104_1004_10004_10


In [26]:
df_f2 = df_f

In [29]:
df_f2['text'] = df_f2['causal_text_w_pairs'].apply(lambda x: re.sub(r'<(/?ARG[01]|/?SIG0)>', '', x))

/var/folders/0z/7ky_dy791xqbq2pvq3wkjlmr0000gn/T/ipykernel_42126/1233164403.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_f2['text'] = df_f2['causal_text_w_pairs'].apply(lambda x: re.sub(r'<(/?ARG[01]|/?SIG0)>', '', x))


In [30]:
df_f2.head()

,causal_text_w_pairs,sent_id,corpus,eg_id,doc_id,index,text
0,The company accused <ARG1>the employee of stea...,10000,cnc,0,train_1000_10000,cnc_train_100_1000_10000_10,The company accused the employee of stealing c...
1,"""<ARG1>The students organized a protest</ARG1>...",10001,cnc,0,train_1001_10001,cnc_train_101_1001_10001_10,"""The students organized a protest because they..."
2,'<ARG1>The students are excited</ARG1> <SIG0>b...,10002,cnc,0,train_1002_10002,cnc_train_102_1002_10002_10,'The students are excited because they won the...
3,'<ARG0>Poor air quality</ARG0> <SIG0>can <ARG1...,10003,cnc,0,train_1003_10003,cnc_train_103_1003_10003_10,'Poor air quality can cause respiratory issues...
4,Since the news about the company's financial d...,10004,cnc,0,train_1004_10004,cnc_train_104_1004_10004_10,Since the news about the company's financial d...


In [31]:
#Always 1 because the data is based on only 1 causal relationship sentences.
df_f2['num_rs'] = 1

/var/folders/0z/7ky_dy791xqbq2pvq3wkjlmr0000gn/T/ipykernel_42126/515156218.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_f2['num_rs'] = 1


In [32]:
#orders the field names, also based on the code from shubhika
df_f2 = df_f2[['corpus', 'doc_id', 'sent_id', 'eg_id', 'index', 'text', 'causal_text_w_pairs', 'num_rs']]

In [33]:
df_f2.to_csv('CNC_chatgpt_output.csv', index=None)